In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

In [4]:
config=BertConfig.from_json_file('fastformer.json')

In [6]:
model = FastformerEncoder(config)

In [8]:
batch_size = 8
seq_len = 50 
dummy_input = torch.randn(batch_size, seq_len, config.hidden_size)
dummy_mask = torch.ones(batch_size, seq_len)

In [9]:
model.eval()
with torch.no_grad():
    output = model(dummy_input, dummy_mask)

In [11]:
print(dummy_input.shape)
print(output.shape)

torch.Size([8, 50, 256])
torch.Size([8, 256])


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\madmo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
tokenizer("Hello my name is", return_tensors='pt')

{'input_ids': tensor([[ 101, 7592, 2026, 2171, 2003,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [15]:
print(f"Token: {tokenizer.pad_token} | ID: {tokenizer.pad_token_id}")

Token: [PAD] | ID: 0
